# SOLUCIONES - SESIÓN 1: Asincronía y Consumo de APIs Externas

**Contenido:**
- Código funcional para cada ejercicio
- Comentarios explicativos
- Notas clave sobre conceptos importantes
- Ejemplos de ejecución y pruebas

## CONFIGURACIÓN DEL ENTORNO

Ejecuta estas celdas para preparar el entorno de trabajo.

In [ ]:
# Instalación de dependencias
!pip install fastapi==0.115.0 httpx==0.27.0 uvicorn[standard]==0.32.0 -q
print("Dependencias instaladas")

In [ ]:
# Importaciones globales
import asyncio
import time
from datetime import datetime
from fastapi import FastAPI, HTTPException, BackgroundTasks
from fastapi.testclient import TestClient
from pydantic import BaseModel, HttpUrl, Field, field_validator
import httpx

print("Imports completados")

---

## EJERCICIO 1: Primera función asíncrona

**Objetivo:** Crear una función asíncrona básica que simule una operación que tarda tiempo.

In [ ]:
# Definición de la función asíncrona
async def operacion_lenta():
    """Simula una operación que tarda 2 segundos"""
    await asyncio.sleep(2)
    return "Operación completada"

# Ejecución de la función
resultado = await operacion_lenta()
print(resultado)

### Explicación

**Elementos clave de esta solución:**

1. **`async def`**: Define una función asíncrona (corutina)
2. **`await asyncio.sleep(2)`**: Suspende la ejecución 2 segundos SIN bloquear el event loop
3. **`await`**: Solo se puede usar dentro de funciones `async def`

**Diferencia con código síncrono:**
- `time.sleep(2)` → Bloquea TODO el programa
- `await asyncio.sleep(2)` → Solo suspende esta corutina, otras pueden ejecutarse

En Jupyter notebooks, usamos directamente `await` en las celdas porque el entorno ya está corriendo un event loop.

---

## EJERCICIO 2: Múltiples tareas concurrentes

**Objetivo:** Ejecutar varias corutinas en paralelo y medir el tiempo total.

In [ ]:
# Definición de la tarea asíncrona
async def tarea(nombre: str, duracion: int):
    """Simula una tarea con duración específica"""
    print(f"Iniciando {nombre}")
    await asyncio.sleep(duracion)
    print(f"Finalizando {nombre}")
    return nombre

# Función principal que ejecuta las tareas en paralelo
async def ejecutar_tareas():
    inicio = time.time()
    
    # asyncio.gather() ejecuta todas las corutinas EN PARALELO
    resultados = await asyncio.gather(
        tarea("A", 2),
        tarea("B", 1),
        tarea("C", 3)
    )
    
    tiempo_total = time.time() - inicio
    print(f"\nTiempo total: {tiempo_total:.2f} segundos")
    print(f"Resultados: {resultados}")

# Ejecutar
await ejecutar_tareas()

### 💡 Nota Clave: ¿Por qué el tiempo total es ~3 segundos y no 6?

**Ejecución paralela con `asyncio.gather()`:**

Las tres tareas se lanzan **simultáneamente**, no secuencialmente:
- Tarea A (2s) y Tarea B (1s) y Tarea C (3s) arrancan al mismo tiempo
- El tiempo total es el de **la tarea más lenta** (~3 segundos)
- NO es la suma de todas (2+1+3 = 6 segundos)

**Analogía:**
Si pones tres ollas al fuego simultáneamente (una tarda 2 min, otra 1 min, otra 3 min), la comida está lista en 3 minutos, no en 6.

**Salida esperada:**
```
Iniciando A
Iniciando B
Iniciando C
Finalizando B       # Termina primero (1s)
Finalizando A       # Termina segundo (2s)
Finalizando C       # Termina último (3s)
Tiempo total: 3.00 segundos
```

Este es el **poder de la asincronía**: maximizar el uso del tiempo de espera.

---

## EJERCICIO 3: Simulación de operaciones I/O

**Objetivo:** Comparar ejecución síncrona vs asíncrona de operaciones I/O.

In [ ]:
# Versión SÍNCRONA (bloqueante)
def consultar_db_sync():
    """Simula consulta bloqueante a base de datos"""
    time.sleep(1)  # BLOQUEA el programa completo
    return {"usuario": "Juan", "edad": 30}

# Versión ASÍNCRONA (no bloqueante)
async def consultar_db_async():
    """Simula consulta no bloqueante a base de datos"""
    await asyncio.sleep(1)  # NO BLOQUEA, suspende solo esta corutina
    return {"usuario": "Juan", "edad": 30}

# Medición SÍNCRONA (3 consultas secuenciales)
print("=== Ejecución SÍNCRONA ===")
inicio = time.time()
for i in range(3):
    resultado = consultar_db_sync()
tiempo_sync = time.time() - inicio
print(f"Tiempo síncrono: {tiempo_sync:.2f}s")

# Medición ASÍNCRONA (3 consultas paralelas)
print("\n=== Ejecución ASÍNCRONA ===")
async def ejecutar_async():
    inicio = time.time()
    
    # Las 3 consultas se ejecutan EN PARALELO
    resultados = await asyncio.gather(
        consultar_db_async(),
        consultar_db_async(),
        consultar_db_async()
    )
    
    tiempo_async = time.time() - inicio
    print(f"Tiempo asíncrono: {tiempo_async:.2f}s")
    
    # Comparación
    print(f"\nMejora: {tiempo_sync / tiempo_async:.1f}x más rápido")

await ejecutar_async()

### Explicación: ¿Cuándo usar asincronía?

**Resultado esperado:**
- Tiempo síncrono: ~3 segundos (1+1+1)
- Tiempo asíncrono: ~1 segundo (todas en paralelo)
- Mejora: 3x más rápido

**Cuándo SÍ usar asincronía:**
- Consultas a bases de datos
- Llamadas HTTP a APIs externas
- Lectura/escritura de archivos
- Operaciones de red (websockets, streaming)

**Cuándo NO usar asincronía:**
- Cálculos matemáticos intensivos (CPU-bound)
- Procesamiento de imágenes/video
- Compresión de datos
- Cualquier operación que NO espera I/O

**Regla de oro:** Si tu código pasa más tiempo esperando (red, disco, DB) que calculando, usa asincronía.

---

## EJERCICIO 4: Endpoint GET asíncrono

**Objetivo:** Crear un endpoint FastAPI asíncrono que retorne datos después de una simulación de consulta.

In [ ]:
# Crear aplicación FastAPI
app = FastAPI(title="Ejercicio 4")

# Endpoint GET asíncrono
@app.get("/productos")
async def listar_productos():
    """Retorna lista de productos tras simular consulta a DB"""
    
    # Simular consulta a base de datos
    await asyncio.sleep(0.5)
    
    # Retornar datos
    return [
        {"id": 1, "nombre": "Laptop", "precio": 999.99},
        {"id": 2, "nombre": "Mouse", "precio": 25.50}
    ]

# Prueba con TestClient
client = TestClient(app)
response = client.get("/productos")

print(f"Status code: {response.status_code}")
print(f"Respuesta: {response.json()}")

### Explicación: Endpoints síncronos vs asíncronos

**¿Cuándo usar `async def` en un endpoint?**

**Usa `async def` si tu endpoint:**
- Consulta una base de datos con driver asíncrono
- Llama a APIs externas con `httpx.AsyncClient()`
- Lee/escribe archivos de forma asíncrona
- Usa cualquier librería que retorne `awaitable`

**Usa `def` normal si tu endpoint:**
- Solo hace cálculos/procesamiento
- Usa librerías síncronas (ej: `requests`)
- No tiene operaciones I/O

**Ventaja en producción:**
FastAPI puede manejar MILES de peticiones concurrentes con endpoints async, pero solo DECENAS con endpoints síncronos (porque no bloquean el event loop).

**Importante:** `TestClient` funciona igual con endpoints sync o async.

---

## EJERCICIO 5: Endpoint POST con validación

**Objetivo:** Crear un endpoint que reciba y valide datos de un nuevo usuario.

In [ ]:
# Modelo Pydantic con validaciones
class UsuarioCreate(BaseModel):
    nombre: str = Field(..., min_length=3, description="Nombre del usuario")
    email: str = Field(..., description="Email del usuario")
    edad: int = Field(..., ge=18, le=100, description="Edad entre 18 y 100")
    
    # Validador de email (Pydantic v2)
    @field_validator('email')
    @classmethod
    def validar_email(cls, v: str) -> str:
        if '@' not in v or '.' not in v:
            raise ValueError('Email inválido')
        return v.lower()  # Normalizar a minúsculas

# Crear aplicación
app = FastAPI(title="Ejercicio 5")

# Endpoint POST
@app.post("/usuarios")
async def crear_usuario(usuario: UsuarioCreate):
    """Crea un nuevo usuario tras validación y simulación de guardado"""
    
    # Simular guardado en base de datos
    await asyncio.sleep(0.3)
    
    # Generar ID y retornar usuario creado
    return {
        "id": 1,
        "nombre": usuario.nombre,
        "email": usuario.email,
        "edad": usuario.edad
    }

# Pruebas
client = TestClient(app)

# Caso VÁLIDO
print("=== Caso válido ===")
usuario_valido = {
    "nombre": "Ana García",
    "email": "ana@ejemplo.com",
    "edad": 25
}
response = client.post("/usuarios", json=usuario_valido)
print(f"Status: {response.status_code}")
print(f"Respuesta: {response.json()}")

# Caso INVÁLIDO (edad < 18)
print("\n=== Caso inválido (edad) ===")
usuario_invalido = {
    "nombre": "Pedro",
    "email": "pedro@ejemplo.com",
    "edad": 16
}
response = client.post("/usuarios", json=usuario_invalido)
print(f"Status: {response.status_code}")
print(f"Error: {response.json()}")

# Caso INVÁLIDO (email)
print("\n=== Caso inválido (email) ===")
usuario_email_malo = {
    "nombre": "Luis",
    "email": "esto-no-es-un-email",
    "edad": 30
}
response = client.post("/usuarios", json=usuario_email_malo)
print(f"Status: {response.status_code}")
print(f"Error: {response.json()}")

### Explicación: Validación automática con Pydantic

**Ventajas de Pydantic en FastAPI:**

1. **Validación automática:** FastAPI valida ANTES de ejecutar tu función
2. **Errores claros:** Si la validación falla, retorna error 422 con detalles
3. **Documentación gratis:** Los campos aparecen automáticamente en Swagger UI
4. **Type hints:** IDE te ayuda con autocompletado

**Diferencias Pydantic v1 vs v2:**

```python
# Pydantic v1 (ANTIGUO)
@validator('email')
def validar_email(cls, v):
    ...

# Pydantic v2 (NUEVO - usado aquí)
@field_validator('email')
@classmethod
def validar_email(cls, v: str) -> str:
    ...
```

**Buena práctica:**
Siempre usa `Field()` con `description=` para documentar tu API automáticamente.

---

## EJERCICIO 6: Manejo de errores con HTTPException

**Objetivo:** Implementar manejo robusto de errores en un endpoint.

In [ ]:
# Base de datos simulada
productos_db = {
    1: {"nombre": "Laptop", "stock": 5},
    2: {"nombre": "Mouse", "stock": 0}
}

# Crear aplicación
app = FastAPI(title="Ejercicio 6")

# Endpoint con manejo de errores
@app.get("/productos/{producto_id}")
async def obtener_producto(producto_id: int):
    """Obtiene un producto validando su existencia y stock"""
    
    # Verificar si el producto existe
    if producto_id not in productos_db:
        raise HTTPException(
            status_code=404,
            detail="Producto no encontrado"
        )
    
    producto = productos_db[producto_id]
    
    # Verificar stock
    if producto["stock"] == 0:
        raise HTTPException(
            status_code=400,
            detail="Producto sin stock"
        )
    
    # Retornar producto si todo está OK
    return {
        "id": producto_id,
        "nombre": producto["nombre"],
        "stock": producto["stock"]
    }

# Pruebas de los 3 casos
client = TestClient(app)

# Caso 1: Producto existe con stock
print("=== Caso 1: Producto con stock ===")
response = client.get("/productos/1")
print(f"Status: {response.status_code}")
print(f"Respuesta: {response.json()}")

# Caso 2: Producto sin stock
print("\n=== Caso 2: Producto sin stock ===")
response = client.get("/productos/2")
print(f"Status: {response.status_code}")
print(f"Error: {response.json()}")

# Caso 3: Producto no existe
print("\n=== Caso 3: Producto no existe ===")
response = client.get("/productos/999")
print(f"Status: {response.status_code}")
print(f"Error: {response.json()}")

### 💡 Nota Clave: HTTPException detiene la ejecución INMEDIATAMENTE

**Comportamiento de `raise HTTPException`:**

Cuando lanzas una `HTTPException`, el código que está DESPUÉS no se ejecuta:

```python
@app.get("/ejemplo")
async def ejemplo():
    print("1. Esto se ejecuta")
    
    raise HTTPException(404, "Error")
    
    print("2. Esto NUNCA se ejecuta")
    return {"mensaje": "3. Esto tampoco"}  # Nunca llega aquí
```

**Ventajas:**
- Código más limpio (no necesitas múltiples `return`)
- Control de flujo claro
- FastAPI captura la excepción y formatea la respuesta automáticamente

**Códigos de estado comunes:**
- `400`: Bad Request (datos inválidos del cliente)
- `404`: Not Found (recurso no existe)
- `500`: Internal Server Error (error del servidor)
- `401`: Unauthorized (autenticación requerida)
- `403`: Forbidden (sin permisos)

**Patrón recomendado:**
Valida primero (raise exceptions), procesa después (return response).

---

## EJERCICIO 7: Cliente httpx básico

**Objetivo:** Consumir una API pública usando httpx de forma asíncrona.

In [ ]:
# Función para obtener usuario de API externa
async def obtener_usuario(user_id: int):
    """Consume API JSONPlaceholder para obtener datos de usuario"""
    
    url = f"https://jsonplaceholder.typicode.com/users/{user_id}"
    
    # Usar AsyncClient como context manager (se cierra automáticamente)
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        
        # Verificar status code
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Error {response.status_code}: Usuario no encontrado")

# Prueba con usuario VÁLIDO
print("=== Usuario válido (ID=1) ===")
try:
    usuario = await obtener_usuario(1)
    print(f"Nombre: {usuario['name']}")
    print(f"Email: {usuario['email']}")
    print(f"Ciudad: {usuario['address']['city']}")
except Exception as e:
    print(f"Error: {e}")

# Prueba con usuario INVÁLIDO
print("\n=== Usuario inválido (ID=999) ===")
try:
    usuario = await obtener_usuario(999)
    print(f"Usuario: {usuario}")
except Exception as e:
    print(f"Error capturado: {e}")

### Explicación: httpx vs requests

**¿Por qué usar `httpx` en lugar de `requests`?**

| Característica | `requests` | `httpx` |
|----------------|------------|----------|
| Asíncrono | ❌ No | ✅ Sí (`AsyncClient`) |
| HTTP/2 | ❌ No | ✅ Sí |
| Timeouts | Manual | Por defecto (5s) |
| API | Síncrona | Sync + Async |

**Uso correcto de AsyncClient:**

```python
# ✅ CORRECTO: context manager
async with httpx.AsyncClient() as client:
    response = await client.get(url)

# ❌ INCORRECTO: sin cerrar conexión
client = httpx.AsyncClient()
response = await client.get(url)
# Conexión queda abierta!
```

**Context manager (`async with`):**
- Abre la conexión al entrar
- Cierra automáticamente al salir (incluso si hay error)
- Libera recursos (sockets, memoria)

**API JSONPlaceholder:**
- API de prueba gratuita
- No requiere autenticación
- Endpoints: `/users`, `/posts`, `/comments`, `/todos`

---

## EJERCICIO 8: Manejo de timeouts y errores de red

**Objetivo:** Implementar manejo robusto de errores en llamadas HTTP.

In [ ]:
# Función con manejo robusto de errores
async def consultar_api_segura(url: str):
    """Consulta una URL con manejo de timeouts y errores de red"""
    
    try:
        # Configurar timeout de 5 segundos
        async with httpx.AsyncClient(timeout=5.0) as client:
            response = await client.get(url)
            
            # Si llegamos aquí, la petición fue exitosa
            return response.json()
            
    except httpx.TimeoutException:
        # La petición tardó más de 5 segundos
        return {"error": "Timeout: El servidor tardó demasiado en responder"}
        
    except httpx.RequestError as e:
        # Error de red (DNS, conexión rechazada, etc.)
        return {"error": f"Error de conexión: {str(e)}"}

# Prueba con URL VÁLIDA
print("=== URL válida ===")
resultado = await consultar_api_segura("https://jsonplaceholder.typicode.com/posts/1")
if "error" in resultado:
    print(f"Error: {resultado['error']}")
else:
    print(f"Título: {resultado.get('title', 'N/A')}")
    print(f"Body: {resultado.get('body', 'N/A')[:50]}...")

# Prueba con URL que causa TIMEOUT
print("\n=== URL con timeout (tarda 10s, límite 5s) ===")
resultado = await consultar_api_segura("https://httpbin.org/delay/10")
print(f"Resultado: {resultado}")

# Prueba con URL INVÁLIDA
print("\n=== URL inválida ===")
resultado = await consultar_api_segura("https://esta-url-no-existe-12345.com")
print(f"Resultado: {resultado}")

### 💡 Nota Clave: La importancia de los timeouts en producción

**¿Por qué son CRÍTICOS los timeouts?**

**Escenario sin timeout:**
```python
# ❌ PELIGROSO en producción
async with httpx.AsyncClient() as client:
    response = await client.get(url)  # Puede esperar PARA SIEMPRE
```

**Consecuencias:**
1. **Bloqueo de workers:** Si una API externa se cuelga, tus workers quedan esperando indefinidamente
2. **Agotamiento de recursos:** Sin timeout, las conexiones no se liberan
3. **Efecto dominó:** Un servicio lento puede tumbar toda tu aplicación

**Ejemplo real:**
Tienes 10 workers de Uvicorn. Si 10 peticiones llaman a una API que no responde (sin timeout), todos tus workers quedan bloqueados. Tu servicio está MUERTO aunque el servidor esté corriendo.

**Timeouts recomendados:**
- **APIs internas:** 2-5 segundos
- **APIs externas:** 5-10 segundos
- **Generación de contenido (IA):** 30-60 segundos

**Configuración avanzada:**
```python
timeout = httpx.Timeout(
    connect=5.0,  # Tiempo máximo para conectar
    read=10.0,    # Tiempo máximo para recibir respuesta
    write=5.0,    # Tiempo máximo para enviar datos
    pool=None     # Sin límite para pool de conexiones
)

async with httpx.AsyncClient(timeout=timeout) as client:
    ...
```

**Regla de oro:** NUNCA hagas peticiones HTTP en producción sin timeout.

---

## EJERCICIO 9: BackgroundTasks simple

**Objetivo:** Usar BackgroundTasks para ejecutar operaciones después de retornar la respuesta.

In [ ]:
# Función que se ejecutará en background
def registrar_log(mensaje: str):
    """Simula escritura de log que tarda 2 segundos"""
    time.sleep(2)  # Bloqueante, pero está en background
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"LOG: {mensaje} - {timestamp}")

# Modelo para la petición
class AccionRequest(BaseModel):
    accion: str

# Crear aplicación
app = FastAPI(title="Ejercicio 9")

# Endpoint que usa BackgroundTasks
@app.post("/acciones")
async def procesar_accion(request: AccionRequest, background_tasks: BackgroundTasks):
    """Procesa una acción y registra en background"""
    
    # Agregar tarea al background (se ejecuta DESPUÉS de retornar respuesta)
    background_tasks.add_task(registrar_log, request.accion)
    
    # Retornar INMEDIATAMENTE (no espera a que termine el log)
    return {"status": "procesando"}

# Prueba del endpoint
client = TestClient(app)

print("Enviando petición...")
inicio = time.time()

response = client.post("/acciones", json={"accion": "Compra realizada"})

tiempo_respuesta = time.time() - inicio
print(f"\nRespuesta recibida en: {tiempo_respuesta:.2f}s")
print(f"Status: {response.status_code}")
print(f"Body: {response.json()}")

print("\n(El log aparecerá ~2 segundos después...)")
time.sleep(3)  # Esperar para ver el log

### Explicación: BackgroundTasks para operaciones diferidas

**¿Qué hace `BackgroundTasks`?**

Ejecuta funciones DESPUÉS de retornar la respuesta al cliente:

```
Cliente hace petición
    ↓
Endpoint procesa y retorna respuesta INMEDIATA
    ↓
Cliente recibe respuesta (< 1 segundo)
    ↓
Background tasks se ejecutan (sin que el cliente espere)
```

**Casos de uso comunes:**
- Enviar emails de confirmación
- Escribir logs a base de datos
- Actualizar estadísticas/métricas
- Enviar notificaciones push
- Limpiar archivos temporales
- Generar reportes PDF

**Ventaja:**
El cliente no espera a que terminen estas operaciones secundarias.

**Importante:**
- Las background tasks pueden ser síncronas (`def`) o asíncronas (`async def`)
- Si falla una background task, NO afecta la respuesta (ya se envió)
- Para tareas LARGAS (>30 segundos), usa Celery o similar

**Limitación:**
Si el servidor se reinicia antes de que termine la tarea, se pierde.

---

## EJERCICIO 10: Sistema integrador completo

**Objetivo:** Integrar todos los conceptos: asincronía, validación, consumo de API externa y BackgroundTasks.

In [ ]:
# Función de background para logging
def registrar_analisis(user_id: int):
    """Registra el análisis de usuario en log"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"LOG: Usuario {user_id} analizado - {timestamp}")

# Función para obtener datos del usuario
async def obtener_datos_usuario(user_id: int):
    """Obtiene información del usuario desde API externa"""
    url = f"https://jsonplaceholder.typicode.com/users/{user_id}"
    
    async with httpx.AsyncClient(timeout=5.0) as client:
        response = await client.get(url)
        
        if response.status_code == 200:
            return response.json()
        else:
            raise HTTPException(404, "Usuario no encontrado en API externa")

# Función para obtener posts del usuario
async def obtener_posts_usuario(user_id: int):
    """Obtiene los posts del usuario desde API externa"""
    url = f"https://jsonplaceholder.typicode.com/posts?userId={user_id}"
    
    async with httpx.AsyncClient(timeout=5.0) as client:
        response = await client.get(url)
        
        if response.status_code == 200:
            return response.json()
        else:
            return []

# Crear aplicación
app = FastAPI(title="Ejercicio 10 - Sistema Integrador")

# Endpoint completo
@app.get("/analizar-usuario/{user_id}")
async def analizar_usuario(user_id: int, background_tasks: BackgroundTasks):
    """
    Analiza un usuario obteniendo sus datos y posts en paralelo.
    Registra el análisis en background.
    """
    
    # 1. VALIDACIÓN de entrada
    if user_id < 1 or user_id > 10:
        raise HTTPException(
            status_code=400,
            detail="ID debe estar entre 1 y 10"
        )
    
    # 2. CONSUMO de APIs externas EN PARALELO
    # gather() ejecuta ambas peticiones simultáneamente
    datos_usuario, posts_usuario = await asyncio.gather(
        obtener_datos_usuario(user_id),
        obtener_posts_usuario(user_id)
    )
    
    # 3. PROCESAMIENTO de datos
    total_posts = len(posts_usuario)
    
    respuesta = {
        "nombre": datos_usuario["name"],
        "email": datos_usuario["email"],
        "total_posts": total_posts
    }
    
    # 4. BACKGROUND TASK para logging
    background_tasks.add_task(registrar_analisis, user_id)
    
    # 5. RETORNAR respuesta inmediata
    return respuesta

# PRUEBAS del sistema completo
client = TestClient(app)

# Caso VÁLIDO
print("=== Prueba con user_id=1 ===")
inicio = time.time()
response = client.get("/analizar-usuario/1")
tiempo = time.time() - inicio

print(f"Status: {response.status_code}")
print(f"Respuesta: {response.json()}")
print(f"Tiempo de respuesta: {tiempo:.2f}s")

# Caso INVÁLIDO (user_id fuera de rango)
print("\n=== Prueba con user_id=20 ===")
response_invalido = client.get("/analizar-usuario/20")
print(f"Status: {response_invalido.status_code}")
print(f"Error: {response_invalido.json()}")

# Esperar para ver el log del background task
print("\n(Esperando log del background task...)")
time.sleep(2)

### 💡 Nota Clave: El poder de `asyncio.gather()` para paralelizar peticiones

**¿Qué hace `gather()` en este ejercicio?**

```python
# EN PARALELO (con gather)
datos_usuario, posts_usuario = await asyncio.gather(
    obtener_datos_usuario(user_id),      # Petición 1
    obtener_posts_usuario(user_id)       # Petición 2
)
# Tiempo total: ~0.5s (el de la petición más lenta)

# VS

# SECUENCIAL (sin gather)
datos_usuario = await obtener_datos_usuario(user_id)  # 0.3s
posts_usuario = await obtener_posts_usuario(user_id)  # 0.2s
# Tiempo total: ~0.5s (suma: 0.3 + 0.2)
```

**En este caso particular:**
Aunque el tiempo es similar, `gather()` reduce la latencia cuando las peticiones tardan más.

**Ejemplo real con latencias mayores:**
Si cada petición tarda 2 segundos:
- Secuencial: 2s + 2s = **4 segundos totales**
- Con gather: max(2s, 2s) = **2 segundos totales**

**Mejora: 50% más rápido**

**Casos de uso ideales para `gather()`:**
- Llamar a múltiples microservicios
- Consultar varias bases de datos
- Obtener datos de diferentes APIs externas
- Cargar múltiples recursos (archivos, imágenes, etc.)

**Limitación:**
Solo funciona si las operaciones son **independientes** (una no depende del resultado de la otra).

**Patrón en producción:**
Este patrón es MUY común en microservicios. Un endpoint de "dashboard" puede necesitar datos de 5-10 servicios diferentes. Con `gather()`, los obtienes todos en paralelo, reduciendo el tiempo de respuesta dramáticamente.

---

## RESUMEN FINAL

Has completado las soluciones de los 10 ejercicios. **Conceptos integrados:**

**Asincronía básica:**
- `async def` para definir corutinas
- `await` para ejecutar operaciones no bloqueantes
- `asyncio.gather()` para ejecución paralela
- `asyncio.sleep()` vs `time.sleep()`

**FastAPI:**
- Endpoints asíncronos con mejor rendimiento
- Validación automática con Pydantic v2
- Manejo de errores con `HTTPException`
- `BackgroundTasks` para operaciones diferidas

**Consumo de APIs:**
- `httpx.AsyncClient()` para peticiones HTTP asíncronas
- Context managers (`async with`) para gestión de recursos
- Timeouts para evitar bloqueos en producción
- Manejo robusto de errores de red

**Patrones de producción:**
- Paralelización con `gather()` para reducir latencia
- Validación temprana (fail-fast)
- Logging en background para no bloquear respuestas
- Configuración de timeouts apropiados

**Siguiente paso:** Practica estos patrones en tus propios proyectos y consulta el resumen de la sesión para referencia rápida.